In [110]:
import pandas as pd
import os

In [122]:
# Specify the directory containing the CSV files
csv_directory = '../../data/source/ilo/'  # Update with the actual path to your directory containing CSV files
country_codes_file = '../../data/source/codes/country-codes.csv'  # Update with the path to your country-codes.csv file
parquet_directory = '../../data/processed/ilo/'  # Update with the path where you want to save Parquet files


In [123]:
# Load the country codes CSV file
country_codes_df = pd.read_csv(country_codes_file)
# Select the necessary columns from the country codes data
country_codes_df = country_codes_df[['ISO3166-1-Alpha-3', 'official_name_en', 'CLDR display name']]
country_codes_df.columns = ['countryCode', 'official_name_en', 'country']


In [124]:
# Define generic transformations
def generic_transformations(df):
    # Example: Standardize sex values
    if 'sex' in df.columns:
        df['sex'] = df['sex'].replace({'Sex: Female': 'Female', 'Sex: Male': 'Male', 'Sex: Total': 'Total'})
    else:
        print("Warning: 'sex' column not found in DataFrame")
    
    # Rename obs_value to value
    if 'obs_value' in df.columns:
        df = df.rename(columns={'obs_value': 'value'})
    else:
        print("Warning: 'obs_value' column not found in DataFrame")
    
    return df



In [130]:
# Define file-specific transformations
def emp_temp_sex_eco_ins_nb_q_transformations(df):
    # Keep specific columns
    df = df[['ref_area.label', 'sex.label', 'time', 'classif1.label', 'classif2.label', 'obs_value']]
    print(f"Columns after keeping specific columns:", df.columns.tolist())
    
    # Rename columns
    df = df.rename(columns={'ref_area.label': 'ref_area', 'sex.label': 'sex', 'classif1.label': 'activity', 'classif2.label': 'sector', 'obs_value': 'value'})
    print(f"Columns after renaming:", df.columns.tolist())
    
    # Apply specific transformations
    df = generic_transformations(df)
    print(f"Columns after generic transformations:", df.columns.tolist())
    
    # Modify content of the 'activity' column
    if 'activity' in df.columns:
        df['activity'] = df['activity'].str.replace('Economic activity ', '', regex=False)
    else:
        print("Warning: 'activity' column not found in DataFrame")
    
    # Modify content of the 'sector' column
    if 'sector' in df.columns:
        df['sector'] = df['sector'].str.replace('Institutional sector: ', '', regex=False)
    else:
        print("Warning: 'sector' column not found in DataFrame")
    
    return df

def emp_temp_sex_ins_nb_a_transformations(df):
    # Keep specific columns
    df = df[['ref_area.label', 'sex.label', 'time', 'classif1.label', 'obs_value']]
    print(f"Columns after keeping specific columns:", df.columns.tolist())
    
    # Rename columns
    df = df.rename(columns={'ref_area.label': 'ref_area', 'sex.label': 'sex', 'classif1.label': 'sector', 'obs_value': 'value'})
    print(f"Columns after renaming:", df.columns.tolist())
    
    # Apply specific transformations
    df = generic_transformations(df)
    print(f"Columns after generic transformations:", df.columns.tolist())
    
    
    # Modify content of the 'sector' column
    if 'sector' in df.columns:
        df['sector'] = df['sector'].str.replace('Institutional sector: ', '', regex=False)
    else:
        print("Warning: 'sector' column not found in DataFrame")
    
    return df

def emp_nifl_sex_rt_a_transformations(df):
    
    # "ref_area.label","source.label","indicator.label","sex.label","time","obs_value","obs_status.label","note_indicator.label","note_source.label"

    # Keep specific columns
    df = df[['ref_area.label', 'sex.label', 'time', 'obs_value']]
    print(f"Columns after keeping specific columns:", df.columns.tolist())
    
    # Rename columns
    df = df.rename(columns={'ref_area.label': 'ref_area', 'sex.label': 'sex', 'obs_value': 'value'})
    print(f"Columns after renaming:", df.columns.tolist())
    
    # Apply specific transformations
    df = generic_transformations(df)
    print(f"Columns after generic transformations:", df.columns.tolist())
       
    return df

# Define transformations for each file
transformations = {
    'EMP_TEMP_SEX_ECO_INS_NB_Q-20240724T2111.csv': emp_temp_sex_eco_ins_nb_q_transformations,
    'EMP_TEMP_SEX_INS_NB_A-20240725T0039.csv': emp_temp_sex_ins_nb_a_transformations,
    'EMP_NIFL_SEX_RT_A-20240726T0000.csv': emp_nifl_sex_rt_a_transformations
    # Add more files and their transformations as needed
}

In [135]:
# Iterate over each CSV file in the directory
for file_name in os.listdir(csv_directory):
    if file_name.endswith('.csv'):
        # Construct full file path
        csv_file_path = os.path.join(csv_directory, file_name)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)
        print(f"Columns in {file_name} after loading:", df.columns.tolist())
        
        # Apply file-specific transformations if defined
        if file_name in transformations:
            transformation_function = transformations[file_name]
            df = transformation_function(df)
            print(f"Columns in {file_name} after specific transformations:", df.columns.tolist())
        else:
            # Apply generic transformations
            df = generic_transformations(df)
            print(f"Columns in {file_name} after generic transformations:", df.columns.tolist())
        
        # Merge the data with the country codes data
        if 'ref_area' in df.columns:
            merged_df = df.merge(country_codes_df, left_on='ref_area', right_on='official_name_en', how='left')
            print(f"Columns in {file_name} after merging:", merged_df.columns.tolist())

            # Drop the 'ref_area' column and select the necessary columns for the final output
            merged_df = merged_df.drop(columns=['ref_area', 'official_name_en'])

            # Construct the Parquet file path
            parquet_file_path = os.path.join(parquet_directory, file_name.replace('.csv', '.parquet'))
            
            # Convert the DataFrame to Parquet format
            merged_df.to_parquet(parquet_file_path, engine='pyarrow')
            
            print(f'Converted and merged {csv_file_path} to {parquet_file_path}')
        else:
            print(f"Error: 'ref_area' column not found in {file_name} after transformations.")

Columns in EMP_NIFL_SEX_RT_A-20240726T0000.csv after loading: ['ref_area.label', 'source.label', 'indicator.label', 'sex.label', 'time', 'obs_value', 'obs_status.label', 'note_indicator.label', 'note_source.label']
Columns after keeping specific columns: ['ref_area.label', 'sex.label', 'time', 'obs_value']
Columns after renaming: ['ref_area', 'sex', 'time', 'value']
Columns after generic transformations: ['ref_area', 'sex', 'time', 'value']
Columns in EMP_NIFL_SEX_RT_A-20240726T0000.csv after specific transformations: ['ref_area', 'sex', 'time', 'value']
Columns in EMP_NIFL_SEX_RT_A-20240726T0000.csv after merging: ['ref_area', 'sex', 'time', 'value', 'countryCode', 'official_name_en', 'country']
Converted and merged ../../data/source/ilo/EMP_NIFL_SEX_RT_A-20240726T0000.csv to ../../data/processed/ilo/EMP_NIFL_SEX_RT_A-20240726T0000.parquet


In [136]:
df2

,ref_area.label,source.label,indicator.label,sex.label,time,obs_value,obs_status.label,note_indicator.label,note_source.label
0,Afghanistan,LFS - Labour Force Survey,Informal employment rate by sex (%),Sex: Total,2021,86.395,NaN,NaN,Repository: ILO-STATISTICS - Micro data proces...
1,Afghanistan,LFS - Labour Force Survey,Informal employment rate by sex (%),Sex: Male,2021,82.987,NaN,NaN,Repository: ILO-STATISTICS - Micro data proces...
2,Afghanistan,LFS - Labour Force Survey,Informal employment rate by sex (%),Sex: Female,2021,97.295,NaN,NaN,Repository: ILO-STATISTICS - Micro data proces...
3,Angola,LFS - Employment Survey,Informal employment rate by sex (%),Sex: Total,2021,90.254,NaN,NaN,Repository: ILO-STATISTICS - Micro data proces...
4,Angola,LFS - Employment Survey,Informal employment rate by sex (%),Sex: Male,2021,84.799,NaN,NaN,Repository: ILO-STATISTICS - Micro data proces...
...,...,...,...,...,...,...,...,...,...
3554,Zimbabwe,LFS - Labour Force Survey,Informal employment rate by sex (%),Sex: Male,2014,83.045,NaN,NaN,Repository: ILO-STATISTICS - Micro data proces...
3555,Zimbabwe,LFS - Labour Force Survey,Informal employment rate by sex (%),Sex: Female,2014,91.551,NaN,NaN,Repository: ILO-STATISTICS - Micro data proces...
3556,Zimbabwe,LFS - Labour Force Survey,Informal employment rate by sex (%),Sex: Total,2011,85.619,Break in series,Break in series: Methodology revised,Repository: ILO-STATISTICS - Micro data proces...
3557,Zimbabwe,LFS - Labour Force Survey,Informal employment rate by sex (%),Sex: Male,2011,79.736,Break in series,Break in series: Methodology revised,Repository: ILO-STATISTICS - Micro data proces...


In [137]:
merged_df

,sex,time,value,countryCode,country
0,Total,2021,86.395,AFG,Afghanistan
1,Male,2021,82.987,AFG,Afghanistan
2,Female,2021,97.295,AFG,Afghanistan
3,Total,2021,90.254,AGO,Angola
4,Male,2021,84.799,AGO,Angola
...,...,...,...,...,...
3554,Male,2014,83.045,ZWE,Zimbabwe
3555,Female,2014,91.551,ZWE,Zimbabwe
3556,Total,2011,85.619,ZWE,Zimbabwe
3557,Male,2011,79.736,ZWE,Zimbabwe
